In [ ]:
# Processing and Displaying run results
import os
import pandas as pd

In [ ]:
# Define a function to extract information from the file content
def extract_info_from_file(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    total_time = None
    final_accuracy = None
    for line in reversed(lines):
        if 'Total training time:' in line:
            total_time = float(line.strip().split(': ')[1].split(' ')[0])
        if 'accuracy:' in line:
            final_accuracy = float(line.strip().split(': ')[1].replace('%', ''))
            break  # Stop after the final accuracy is found as it is the first instance from the end

    return total_time, final_accuracy

# Function to parse file name
def parse_filename(file_name):
    parts = file_name.split('-')
    if len(parts) == 5:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3].split('.')[0]  # Remove file extension, prob need to add run number here
    else:
        optimizer, model, dataset, augmentation = None, None, None, None

    return optimizer, model, dataset, augmentation

In [ ]:
# Specify the directory
folder_path = 'logs'  # Change 'folder_name' to the name of your folder

# DataFrame to store the results
results_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Total Time (s)', 'Final Accuracy (%)'])

results_df

In [ ]:
# Process each file in the directory
for file in os.listdir(folder_path):
    if file.endswith('.out'):  # Check if the file is a .out file
        file_path = os.path.join(folder_path, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        optimizer, model, dataset, augmentation = parse_filename(file)
        
        # Append the results to the DataFrame
        results_df = results_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time
        }, ignore_index=True)

results_df['Total Time (min)'] = results_df['Total Time (s)'] / 60

results_df.head()

In [ ]:
grouped_df = results_df.groupby(['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation']).agg({
    'Final Accuracy (%)': ['mean', 'std'],
    'Total Time (min)': ['mean', 'std']
}).reset_index()

# Rename the columns for clarity
grouped_df.columns = [
    'Optimizer', 
    'Model Architecture', 
    'Dataset', 
    'Data Augmentation', 
    'Final Accuracy Mean (%)', 
    'Final Accuracy Std (%)',
    'Total Time Mean (min)', 
    'Total Time Std (min)' 
]

grouped_df

In [ ]:
df_sorted = grouped_df.sort_values(by=['Dataset', 'Data Augmentation'])

df_sorted

In [ ]:
from tabulate import tabulate

In [ ]:
'''
# Define the custom order for Data Augmentation
augmentation_order = ["basicaug", "cutout", "autoaug", "randaug"]
optimizer_order = ["sgd", "nonaccelgamsgd", "gamsgd"]

# Create a categorical type for Data Augmentation with the specified order
grouped_df["Data Augmentation"] = pd.Categorical(grouped_df["Data Augmentation"], categories=augmentation_order, ordered=True)

# Filter and sort DataFrame to match the desired format
df_sorted = grouped_df.sort_values(by=["Dataset", "Data Augmentation", "Optimizer"])
df_sorted = df_sorted.loc[df_sorted["Data Augmentation"] != "randaug"]
df_cifar10 = df_sorted[df_sorted["Dataset"] == "cifar10"]
df_cifar100 = df_sorted[df_sorted["Dataset"] == "cifar100"]

# Prepare the CIFAR-10 and CIFAR-100 parts of the table
table_cifar10 = df_cifar10.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()
table_cifar100 = df_cifar100.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()

table_cifar10 = table_cifar10[["Data Augmentation"] + optimizer_order]

# Add a new column to distinguish between CIFAR-10 and CIFAR-100
table_cifar10.insert(0, 'Dataset', 'CIFAR-10')
table_cifar100.insert(0, 'Dataset', 'CIFAR-100')

# Combine the tables
table = pd.concat([table_cifar10, table_cifar100], axis=0)

# Print the table
print(tabulate(table, headers='keys', tablefmt='pipe'))
'''

### Gradient Norm Only Minimization (GNOM)

#### GNOM learning rate testing (20 epochs):

In [ ]:
# Specify the directory
folder_path_gnom = 'logs/GNOM-Tests'  


In [ ]:
def extract_epoch_time(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    epoch_time = 0
    count = 0
    for line in lines:
        if 'Epoch' and 'time:' in line:
            count += 1
            epoch_time += float(line.strip().split(': ')[1].split(' ')[0])
    return (epoch_time / count)

def extract_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    epochs = 20
    if len(parts) >= 6:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size = None, None, None, None, None, None
    
    if len(parts) == 7:
        learning_rate = parts[5]
        batch_size = parts[6].split('.')[0] 
        if batch_size == 'full':
            batch_size = 128
            epochs = 200
    else:
        learning_rate = parts[5][:-4]
        batch_size = 128
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

In [ ]:
gnom_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom):
    if '-lr-' in file:  
        file_path = os.path.join(folder_path_gnom, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        epoch_time = extract_epoch_time(file_path)
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_test_info(file_path)
        
        gnom_df = gnom_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Learning Rate': float(learning_rate),
            'Batch Size': int(batch_size),
            'Epochs': epochs,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time,
            'Avg Epoch Time (s)': epoch_time
        }, ignore_index=True)

gnom_df['Total Time (min)'] = gnom_df['Total Time (s)'] / 60

gnom_df_sorted = gnom_df.sort_values(by=['Batch Size', 'Learning Rate'])

gnom_df_sorted

#### GNOM Gradient Norm and Hessian Eigenvalue testing:

In [ ]:
def extract_ev_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    if len(parts) == 9:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
        epochs = parts[7]
    elif len(parts) == 8:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
        epochs = parts[7][:-4]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = None, None, None, None, None, None, None
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

def extract_grad_info(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    grad_norm = None
    top_ev = None
    smallest_ev = None
    for line in reversed(lines):
        if 'Smallest Hessian Eigenvalue:' in line:
            smallest_ev = float(line.strip().split(': ')[1])
        if 'Norm of the Gradient:' in line:
            grad_norm = float(line.strip().split(': ')[1])
        if 'Largest Hessian Eigenvalue:' in line:
            top_ev = float(line.strip().split(': ')[1])
            break

    return grad_norm, top_ev, smallest_ev

In [ ]:
folder_path_gnom_ev = 'logs/GNOM-Saved-Models'  
gnom_ev_df = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)', 'Gradient Norm', 'Largest Eigenvalue of Hessian', 'Smallest Eigenvalue of Hessian'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom_ev):
    file_path = os.path.join(folder_path_gnom_ev, file)
    total_time, final_accuracy = extract_info_from_file(file_path)
    epoch_time = extract_epoch_time(file_path)
    optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_ev_test_info(file_path)
    grad_norm, top_ev, smallest_ev = extract_grad_info(file_path)
    
    gnom_ev_df = gnom_ev_df._append({
        'Optimizer': optimizer,
        'Learning Rate': float(learning_rate),
        'Batch Size': int(batch_size),
        'Epochs': epochs,
        'Final Accuracy (%)': final_accuracy,
        'Total Time (s)': total_time,
        'Avg Epoch Time (s)': epoch_time,
        'Gradient Norm': grad_norm,
        'Largest Eigenvalue of Hessian': top_ev,
        'Smallest Eigenvalue of Hessian': smallest_ev
    }, ignore_index=True)

gnom_ev_df['Total Time (min)'] = gnom_ev_df['Total Time (s)'] / 60
gnom_ev_df['Avg Epoch Time (min)'] = gnom_ev_df['Avg Epoch Time (s)'] / 60
gnom_ev_df.drop(columns=['Total Time (s)', 'Avg Epoch Time (s)'], inplace=True)

gnom_ev_df_sorted = gnom_ev_df.sort_values(by=['Epochs','Optimizer','Batch Size'])

print("All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations")
gnom_ev_df_sorted


### GNOM Noised

In [ ]:
def extract_noise_info(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    batches_noised = None
    for line in reversed(lines):
        if 'Noise applied in' in line:
            batches_noised = float(line.strip().split(' ')[8])
            break

    return batches_noised

def extract_noised_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    if len(parts) >= 8:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
        learning_rate = parts[5]
        batch_size = parts[6]
    
    if len(parts) == 10:
        epochs = parts[7]
        threshold = parts[8]
        radius = parts[9][:-4]
    else:
        epochs = parts[7][:-4]
        threshold, radius = 0.1, 0.01
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs, threshold, radius

In [ ]:
folder_path_gnom_noised = 'logs/GNOM-Noised'  
gnom_noised_df = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)', 'Gradient Norm', 'Largest Eigenvalue of Hessian', 'Smallest Eigenvalue of Hessian'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom_noised):
    file_path = os.path.join(folder_path_gnom_noised, file)
    total_time, final_accuracy = extract_info_from_file(file_path)
    epoch_time = extract_epoch_time(file_path)
    optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs, threshold, radius = extract_noised_test_info(file_path)
    grad_norm, top_ev, smallest_ev = extract_grad_info(file_path)
    batches_noised = extract_noise_info(file_path)
    
    gnom_noised_df = gnom_noised_df._append({
        'Optimizer': optimizer,
        'Learning Rate': float(learning_rate),
        'Batch Size': int(batch_size),
        'Epochs': epochs,
        'Final Accuracy (%)': final_accuracy,
        'Total Time (s)': total_time,
        'Avg Epoch Time (s)': epoch_time,
        'Gradient Norm': grad_norm,
        'Largest Eigenvalue of Hessian': top_ev,
        'Smallest Eigenvalue of Hessian': smallest_ev,
        'Batches Noise was Added (%)': batches_noised,
        'Noise Gradient Norm Threshold': threshold,
        'Noise Added Radius': radius
    }, ignore_index=True)

gnom_noised_df['Total Time (min)'] = gnom_noised_df['Total Time (s)'] / 60
gnom_noised_df['Avg Epoch Time (min)'] = gnom_noised_df['Avg Epoch Time (s)'] / 60
gnom_noised_df.drop(columns=['Total Time (s)', 'Avg Epoch Time (s)'], inplace=True)

gnom_noised_df_sorted = gnom_noised_df.sort_values(by=['Final Accuracy (%)'], ascending=False)

print("All of the below runs were done with ResNet18, CIFAR-10, and basic data augmentations")

gnom_noised_df_sorted

### GNOM Normal vs. Noised

In [ ]:
combined_results = pd.concat([gnom_ev_df_sorted, gnom_noised_df_sorted], ignore_index=True)
gnom_results = combined_results[combined_results['Optimizer'].isin(["gnomsgd"])]
gnom_results = gnom_results.sort_values(by=['Final Accuracy (%)'])
gnom_results

### GNOM Noised with Partitioned Gradient Calculation Dataset
#### Batch fixed at 256 and learning rate fixed at 0.01

In [ ]:
folder_path_gnom_noised_part = 'logs/GNOM-Noised-Partitioned'  
gnom_noised_part_df = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)', 'Gradient Norm', 'Largest Eigenvalue of Hessian', 'Smallest Eigenvalue of Hessian'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom_noised_part):
    file_path = os.path.join(folder_path_gnom_noised_part, file)
    total_time, final_accuracy = extract_info_from_file(file_path)
    epoch_time = extract_epoch_time(file_path)
    optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs, threshold, radius = extract_noised_test_info(file_path)
    grad_norm, top_ev, smallest_ev = extract_grad_info(file_path)
    batches_noised = extract_noise_info(file_path)
    
    gnom_noised_part_df = gnom_noised_part_df._append({
        'Optimizer': optimizer,
        'Learning Rate': float(learning_rate),
        'Batch Size': int(batch_size),
        'Epochs': epochs,
        'Final Accuracy (%)': final_accuracy,
        'Total Time (s)': total_time,
        'Avg Epoch Time (s)': epoch_time,
        'Gradient Norm': grad_norm,
        'Largest Eigenvalue of Hessian': top_ev,
        'Smallest Eigenvalue of Hessian': smallest_ev,
        'Batches Noise was Added (%)': batches_noised,
        'Noise Gradient Norm Threshold': threshold,
        'Noise Added Radius': radius
    }, ignore_index=True)

gnom_noised_part_df['Total Time (min)'] = gnom_noised_part_df['Total Time (s)'] / 60
gnom_noised_part_df['Avg Epoch Time (min)'] = gnom_noised_part_df['Avg Epoch Time (s)'] / 60
gnom_noised_part_df.drop(columns=['Total Time (s)', 'Avg Epoch Time (s)'], inplace=True)

gnom_noised_part_df_sorted = gnom_noised_part_df.sort_values(by=['Final Accuracy (%)'], ascending=False)

gnom_noised_part_df_sorted